In [43]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [44]:
path_img = 'vidCaps/'

In [45]:
all_files = os.listdir('dataset/')

In [65]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs):
        'Initialization'
        self.list_IDs = list_IDs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(len(self.list_IDs))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index:(index+1)]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        PATH_NPY = 'dataset/'
        for ID in list_IDs_temp:
            npy_file = os.path.join(PATH_NPY, ID)
            npy_data = np.load(npy_file, allow_pickle=True)

            X, y = (npy_data[0][0], npy_data[0][1])
            
            y = tf.constant(float(y))
            
        return X, y

In [66]:
# TRAIN SET
all_files_loc = 'dataset/'
all_files = os.listdir(all_files_loc)
print(len(all_files))

20398


In [67]:
train_files = all_files[:18398]
val_files = all_files[18398:20398]

In [68]:
partition = [item for item in train_files]
val_partition = [item for item in val_files]

In [69]:
print(len(partition), len(val_partition))

18398 2000


In [70]:
training_generator = DataGenerator(partition)
validation_generator = DataGenerator(val_partition)

In [71]:
print(len(training_generator))
print(len(validation_generator))

18398
2000


In [72]:
sample_input = np.load('dataset/training_data-183.npy', allow_pickle=True)
sample_input[0][0].shape
y = sample_input[0][1]

In [73]:
ytest = tf.constant(float(y))
print(ytest)
print(type(ytest))

tf.Tensor(24.78153, shape=(), dtype=float32)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [74]:
inputs = keras.Input(shape=(480, 640, 3))

In [75]:
dense = layers.Dense(64, activation='relu')

In [76]:
x = dense(inputs)

In [77]:
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)

In [78]:
model = keras.Model(inputs=inputs, outputs=outputs, name='speed_model')

In [79]:
model.summary()

Model: "speed_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 480, 640, 3)]     0         
_________________________________________________________________
dense_6 (Dense)              (None, 480, 640, 64)      256       
_________________________________________________________________
dense_7 (Dense)              (None, 480, 640, 64)      4160      
_________________________________________________________________
dense_8 (Dense)              (None, 480, 640, 1)       65        
Total params: 4,481
Trainable params: 4,481
Non-trainable params: 0
_________________________________________________________________


In [80]:
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')
optimizer = keras.optimizers.Adam()
metrics = ['accuracy']

In [81]:
model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=metrics
)

In [82]:
hst = model.fit_generator(generator=training_generator, 
                           epochs=200, 
                           validation_data=validation_generator,
                           use_multiprocessing=True,
                           max_queue_size=32)

Epoch 1/200


ValueError: in user code:

    /home/andrew/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/andrew/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/andrew/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/andrew/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/andrew/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /home/andrew/anaconda3/e